In [ ]:
# Mandatory imports
import requests
import getpass
from typing import TypedDict, List, Any

In [ ]:
# Helper functions (this cell could be inserted in every notebook)

global projectId, apiKey, baseUrl

_projectId: str | None = None
_apiKey: str | None = None
_baseUrl: str = 'https://api.jinko.ai'

class CoreItemId(TypedDict):
  id: str
  snapshotId: str

def getHeaders() -> dict[str, str]:
  return {
  'X-jinko-project-id': _projectId,
  'Authorization': 'ApiKey ' + _apiKey
}

def makeUrl(path: str):
  return _baseUrl + path

def makeRequest(path: str, method: str ='GET', json=None):
  response = requests.request(method, _baseUrl + path, headers=getHeaders(), json=json)
  if response.status_code != 200:
    print(response.json())
    response.raise_for_status()
  return response

def checkAuthentication() -> bool:
  response = requests.get(makeUrl('/app/v1/project-item/?first=1'), headers=getHeaders())
  if response.status_code == 401:
    return False
  if response.status_code != 200:
    print(response.json())
    response.raise_for_status()
  return True

# Ask user for API key/projectId and check authentication
def initialize(projectId: str | None = None, baseUrl: str | None = None):
   global _projectId, _apiKey, _baseUrl
   if baseUrl is not None:
     _baseUrl = baseUrl

   if _apiKey is None:
     _apiKey = getpass.getpass('Please enter your API key')
   if _apiKey == '':
     message = 'API key cannot be empty'
     print(message)
     raise Exception(message)

   _projectId = projectId
   if _projectId is None:
     _projectId = getpass.getpass('Please enter your Project Id')
   if _projectId == '':
     message = 'Project Id cannot be empty'
     print(message)
     raise Exception(message)

   if not checkAuthentication():
     message = 'Authentication failed for Project "%s"' % (projectId)
     print(message)
     raise Exception(message)
   print('Authentication successful')

def getProjectItem(shortId: str, revision: int | None = None):
    response = makeRequest('/app/v1/project-item')
    projectItems = response.json()
    item = next(filter(lambda item: item['sid'] == shortId, projectItems), None)
    if item is None:       
       message = 'ProjectItem "%s"' % (shortId)
       if revision is not None:
         message += ' (revision %s)' % (str(revision))
       message += ' not found in Project "%s"' % (projectId)
       print(message)
       raise Exception(message)
    return item

def getCoreItemId(shortId: str, revision: int | None = None) -> CoreItemId:
    item = getProjectItem(shortId, revision)
    if 'coreItemId' not in item or item['coreItemId'] is None:
       message = 'ProjectItem "%s" has no CoreItemId' % (shortId)
       print(message)
       raise Exception(message)
    return item['coreItemId']

In [ ]:
# Configuration

# Fill your Project Id (ex: '14495063-918c-441a-a898-3131d70b02b0')
# If not set, you will be asked for it interactively
projectId = None

# This function asks for your API key interactively
# It will ensure that authentication is correct
# It it also possible to override the base url by passing baseUrl=...
initialize(projectId)

In [ ]:
import zipfile
import io

# Fill the short Id of your Trial (ex: tr-EKRx-3HRt)
trialId = 'tr-EKRx-3HRt'
if trialId is None:
  raise Exception('Please specify a Trial Id')

# Convert short Id to coreItemId
coreItemId = getCoreItemId(trialId, 1)

# List all Trial versions
versions: List[Any] = makeRequest('/core/v2/trial_manager/trial/%s/status'%(coreItemId['id'])).json()

# Get the latest completed version
latestCompletedVersion = next((item for item in versions if item['status'] == 'completed'), None)
if latestCompletedVersion is None:
  raise Exception('No completed Trial version found')

# Retrieve scalar results
response = makeRequest('/core/v2/result_manager/scalars_summary', method='POST', json={
  "select": [
    "x-max-on-medium-minus-its-control",
  ],
  "trialId": latestCompletedVersion['simulationId']
})
archive = zipfile.ZipFile(io.BytesIO(response.content))
filename = archive.namelist()[0]
csv = archive.read(filename).decode('utf-8')

print(csv)